<center><h1> Function to extract ED412_ADJUSTED profile from Temperature, Salinity profiles and corresponding Satellite Ocean Color Matchups </h1></center>



<div style="text-align: right"><i> Renosh - Nov 2023 </i></div>
<div style="text-align: right"><i> Email: renosh.pr@imev-mer.fr </i></div>
<div style="text-align: right"><i> Email: pr.renosh@gmail.com </i></div>

In [3]:
import warnings
warnings.filterwarnings('ignore')

## 1) Load all libraries

In [7]:
#######################################################################################
#######################################################################################
# Load all the libraries
import sys
import pathlib
from pathlib import Path
import pickle
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset as NetCDFFile 
import import_ipynb
from datetime import date, datetime, timedelta
import datetime as dt
# import oceans.sw_extras as swe
import gsw
import matplotlib.pyplot as plt
import timezonefinder, pytz
tf = timezonefinder.TimezoneFinder()
#######################################################################################
#######################################################################################

## 2) Light model paths

In [8]:
#######################################################################################
#######################################################################################

# MODEL PATH---->

# get the current working directory
curr_work_dir = Path.cwd()
# home_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
# print output to the console
# print(curr_work_dir)
MODEL_PATH = (''.join([str(curr_work_dir),'/MODELS/ED412_2023_MODELS']))
# MODEL_PATH = (''.join([str(home_dir),'/MODELS/PAR_2023_MODELS']))

#######################################################################################
# Lad the PCAs model
# No need to change anything in this section by the user
pkl_filename_temp_pca = ''.join([str(MODEL_PATH),str("/temp_pca_51.pkl")])
pca_temp = pickle.load(open(pkl_filename_temp_pca, "rb"))

pkl_filename_sal_pca = ''.join([str(MODEL_PATH),str("/sal_pca_51.pkl")])
pca_sal = pickle.load(open(pkl_filename_sal_pca, "rb"))

#######################################################################################
# Load the x-scaler from the folder 
# No need to change anything in this section by the user
pkl_filename_x_scaler = ''.join([str(MODEL_PATH),str("/x_scaler.pkl")]) 
x_scaler = pickle.load(open(pkl_filename_x_scaler, 'rb')) 
# Load the x-scaler from the file 
pkl_filename_y_scaler = ''.join([str(MODEL_PATH),str("/y_scaler.pkl")])
y_scaler = pickle.load(open(pkl_filename_y_scaler, 'rb'))
#######################################################################################
#######################################################################################


## 3) Light model Functions

In [1]:
#######################################################################################
#######################################################################################
# No need to change anything in this section by the user
# CREATE SCALED INPUTS
def INPUTS_ED412_ADJUSTED_2023(PAR, MLD, ZPD_MOREL, RHO_WN_412, RHO_WN_443, RHO_WN_490, RHO_WN_555, RHO_WN_670,
                               sin_doy, cos_doy, sin_LC_hour, cos_LC_hour, temp_cmems, sal_cmems, depths_cmems, depth_output):
                          
    
    INPUTS1 = [PAR, MLD, ZPD_MOREL, RHO_WN_412, RHO_WN_443, RHO_WN_490, RHO_WN_555, RHO_WN_670,
              sin_doy, cos_doy, sin_LC_hour, cos_LC_hour]
    
    temp_51 = np.interp(depth_output, depths_cmems, temp_cmems) # made into 51 depths
    temp_51 = temp_51.reshape(-1, 1)
    sal_51 = np.interp(depth_output, depths_cmems, sal_cmems) # made into 51 depths
    sal_51 = sal_51.reshape(-1, 1) 
    
       

    # Apply PCA on temperature, salinity and spiciness profiles to get the principal components
    PrincipalComponentsTemp_Test = pca_temp.transform(np.transpose(temp_51))[:,:5]
    principalComponentsSal_Test = pca_sal.transform(np.transpose(sal_51))[:,:4]   

         
    INPUTS = INPUTS1 + np.concatenate(PrincipalComponentsTemp_Test).ravel().tolist() + np.concatenate(principalComponentsSal_Test).ravel().tolist()
    

    input_features = ['PAR','MLD', 'ZPD_MOREL', 'RHO_WN_412','RHO_WN_443','RHO_WN_490','RHO_WN_555', 'RHO_WN_670',
                      'sin_doy', 'cos_doy','sin_LC_hour', 'cos_LC_hour',
#                       'x_cart', 'y_cart', 'z_cart',
                      'TEMP_PC_N_0', 'TEMP_PC_N_1', 'TEMP_PC_N_2', 'TEMP_PC_N_3','TEMP_PC_N_4',
                      'SAL_PC_N_0', 'SAL_PC_N_1', 'SAL_PC_N_2','SAL_PC_N_3']

    INPUTS = pd.DataFrame(INPUTS)
    INPUTS = INPUTS.transpose()    
    INPUTS.columns = input_features
    INPUTS_scaled = pd.DataFrame(x_scaler.transform(INPUTS), columns=INPUTS.columns)
    
    return INPUTS_scaled
#######################################################################################
# No need to change anything in this section by the user
# FUNCTION to derive ED412_ADJUSTED profile
def SOCA_ED412_ADJUSTED_2023(INPUTS_ED412_ADJUSTED_2023,depth_output):
    
    n_sample = INPUTS_ED412_ADJUSTED_2023.shape[0]
    YPRED_TOTAL = pd.DataFrame()
    # YPRED_KD_TOTAL = pd.DataFrame()
    nc1 = [70,98,103,115,120,134,135,139,142,143]    
    # nc2 = [59,62,16,33,15,46,34,36,77,48]
    
    for x in np.arange(len(nc1)):
        pkl_filename = ''.join([str(MODEL_PATH),str("/pickle_model_ED412_2023_"),str(x),str(".pkl")])
        MLP = pickle.load(open(pkl_filename, 'rb'))
 
        # Apply MLP to the test dataset
        ypred_scaled = MLP.predict(INPUTS_ED412_ADJUSTED_2023)
        
        # Detransform the outputs:
        ypred = pd.DataFrame(
            y_scaler.inverse_transform(ypred_scaled),
#             columns=np.arange(25),
            columns=np.arange(51),
            index=np.arange(n_sample)
        )

        # make vectors
        ED412_PRED = ypred 
        ED412_PRED_vec = ED412_PRED.values.ravel()  
        ED412_PRED_vec = pow(10,ED412_PRED_vec)        
#         ypred_vec = ypred.values.ravel()
        YPRED_TOTAL = pd.concat([YPRED_TOTAL, pd.DataFrame(ED412_PRED_vec)], axis=1)
        #       
   
       
    soca_ed412=np.mean(YPRED_TOTAL, axis=1)
    soca_ed412_err=np.std(YPRED_TOTAL, axis=1)    
   
    return soca_ed412, soca_ed412_err
#######################################################################################
# Function to derive ED412_ADJUSTED Profiles
# No need to change anything in this section by the user
# This function call 2 functions INPUTS_ED412_ADJUSTED_2023 & SOCA_ED412_ADJUSTED_2023
def ED412_ADJUSTED_profiles(PAR, MLD, ZPD, RHO_WN_412, RHO_WN_443, RHO_WN_490, RHO_WN_555, RHO_WN_670,
                                                        sin_doy, cos_doy, sin_LC_hour, cos_LC_hour, temp, sal, depths_argo, depth_output):
    
    soca_ed412_adjusted_inputs = INPUTS_ED412_ADJUSTED_2023(PAR, MLD, ZPD, RHO_WN_412, RHO_WN_443, RHO_WN_490, RHO_WN_555, RHO_WN_670,
                                                        sin_doy, cos_doy, sin_LC_hour, cos_LC_hour, temp, sal, depths_argo, depth_output)
    
    ed412_soca =  SOCA_ED412_ADJUSTED_2023(soca_ed412_adjusted_inputs,depth_output)[0]   
    ed412_soca_err =  SOCA_ED412_ADJUSTED_2023(soca_ed412_adjusted_inputs,depth_output)[1]
    
    return ed412_soca, ed412_soca_err   
#######################################################################################
#######################################################################################
